# Install necessary packages

## Pre-requisities

In [ ]:
#requirements.txt 
!pip freeze

In [2]:
import warnings
from IPython.display import HTML, display, Markdown, clear_output
import random
import logging

import os
import getpass
import platform
import json
import requests
from datetime import datetime
from pytz import timezone
import time

warnings.filterwarnings('ignore')

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

#Setting random seed - 41 in this case
random.seed(41)

display(Markdown('<span style="color:darkgreen; font-style: italic; font-size: 15px">Prerequisite Code #1 for <b>suppressing warning</b> is EXECUTED!</span>'))

<span style="color:darkgreen; font-style: italic; font-size: 15px">Prerequisite Code #1 for <b>suppressing warning</b> is EXECUTED!</span>

### Packges needed for data cleaning and pre-processing ###

In [3]:
# Import packages
import pandas as pd
from sklearn.preprocessing import LabelEncoder          
from sklearn.model_selection import train_test_split  
import matplotlib.pyplot as plt 
import numpy as np
import copy
import IPython
import seaborn as sns
# import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from termcolor import colored
import re
import sympy

## Required for pre-processing and modelling

In [4]:
from sklearn.preprocessing import OneHotEncoder
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from imblearn.over_sampling import SMOTE
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_curve
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lars

from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.metrics import precision_recall_curve, accuracy_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score

# Import Dataset

In [5]:
data = pd.read_excel('Data.xlsx', sheet_name='Dataset')

# Data Understanding

### Quick Peek

In [7]:
display(Markdown(""" <span style="color:darkgreen; font-style:italic; font-size:15px">  <b>First few rows of the dataset is shown below   </b></span>"""))
data.head(5)

 <span style="color:darkgreen; font-style:italic; font-size:15px">  <b>First few rows of the dataset is shown below   </b></span>

,Ptid,Persistency_Flag,Gender,Race,Ethnicity,Region,Age,Speciality,Gluco_Prior_Ntm,Gluco_During_Rx,...,Risk_Family_History_Of_Osteoporosis,Risk_Low_Calcium_Intake,Risk_Vitamin_D_Insufficiency,Risk_Poor_Health_Frailty,Risk_Excessive_Thinness,Risk_Hysterectomy_Oophorectomy,Risk_Estrogen_Deficiency,Risk_Immobilization,Risk_Recurring_Falls,Count_Of_Risks
0,P701,Non-Persistent,Female,Caucasian,Not Hispanic,South,>75,GENERAL PRACTITIONER,N,N,...,N,N,N,N,N,N,N,N,N,0
1,P702,Non-Persistent,Female,Caucasian,Not Hispanic,South,55-65,GENERAL PRACTITIONER,N,N,...,N,N,N,N,N,N,N,N,N,0
2,P703,Non-Persistent,Female,Caucasian,Hispanic,Midwest,55-65,GENERAL PRACTITIONER,N,N,...,N,Y,N,N,N,N,N,N,N,2
3,P704,Persistent,Female,Caucasian,Not Hispanic,Midwest,65-75,GENERAL PRACTITIONER,N,Y,...,N,N,N,N,N,N,N,N,N,1
4,P705,Persistent,Female,Caucasian,Not Hispanic,Other/Unknown,55-65,GENERAL PRACTITIONER,Y,Y,...,N,N,N,N,N,N,N,N,N,1


### Formatting

In [8]:
original_cols = data.columns
cols_vis = ' || '.join(original_cols)
print(colored("\nColumn Names (Before Formatting):", 'magenta', attrs = ['bold']),"\n{}".format(cols_vis))

#List of special characters that should not be present
special_chars = r'[?|$|#|@|%|*|(|)|<|>|:|"|{|}|,|.|;|!|^|\|]'

#Lower case and remove white spaces
data.columns = list(map(lambda x:re.sub(special_chars,r'',x.lower().replace(' ','_').replace("'","").replace("|","_")), data.columns))

updated_cols = data.columns
cols_vis = ' || '.join(updated_cols)
print(colored("\nColumn Names (After Formatting):", 'blue', attrs = ['bold']), "\n{}".format(cols_vis))


Column Names (Before Formatting): 
Ptid || Persistency_Flag || Gender || Race || Ethnicity || Region || Age || Speciality || Gluco_Prior_Ntm || Gluco_During_Rx || Dexa_Freq_During_Rx || Dexa_During_Rx || Frag_Frac_Prior_Ntm || Frag_Frac_During_Rx || Risk_Segment_Prior_Ntm || Tscore_Bucket_Prior_Ntm || Risk_Segment_During_Rx || Tscore_Bucket_During_Rx || Change_T_Score || Change_Risk_Segment || Injectable_Experience_During_Rx || Comorb_Encounter_For_Screening_For_Malignant_Neoplasms || Comorb_Encounter_For_Immunization || Comorb_Encntr_For_General_Exam_W_O_Complaint,_Susp_Or_Reprtd_Dx || Comorb_Vitamin_D_Deficiency || Comorb_Other_Joint_Disorder_Not_Elsewhere_Classified || Comorb_Encntr_For_Oth_Sp_Exam_W_O_Complaint_Suspected_Or_Reprtd_Dx || Comorb_Long_Term_Current_Drug_Therapy || Comorb_Dorsalgia || Comorb_Personal_History_Of_Other_Diseases_And_Conditions || Comorb_Other_Disorders_Of_Bone_Density_And_Structure || Comorb_Disorders_of_lipoprotein_metabolism_and_other_lipidemias || Como

## Data Summary

In [9]:
display(Markdown(""" <span style="color:darkgreen; font-style:italic; font-size:15px">  <b>Data type of the loaded dataset is shown below   </b></span>"""))
display(data.dtypes)

cols = data.columns
#Getting the list of numerical features
num_cols = data._get_numeric_data().columns.tolist()
num_cols_vis = ' || '.join(num_cols)
print(colored("Numerical columns:\nCount:", 'magenta', attrs=['bold']),"{}\n{}".format(len(num_cols),num_cols_vis))

#Getting list of categorical features
cat_cols = list(set(cols) - set(num_cols))
cat_cols_vis = ' || '.join(cat_cols)
print(colored("Categorical columns:\nCount:", 'magenta', attrs=['bold']),"{}\n{}".format(len(cat_cols),cat_cols_vis))

 <span style="color:darkgreen; font-style:italic; font-size:15px">  <b>Data type of the loaded dataset is shown below   </b></span>

ptid                              object
persistency_flag                  object
gender                            object
race                              object
ethnicity                         object
                                   ...  
risk_hysterectomy_oophorectomy    object
risk_estrogen_deficiency          object
risk_immobilization               object
risk_recurring_falls              object
count_of_risks                     int64
Length: 65, dtype: object

Numerical columns:
Count: 2
dexa_freq_during_rx || count_of_risks
Categorical columns:
Count: 63
risk_excessive_thinness || risk_vitamin_d_insufficiency || comorb_vitamin_d_deficiency || concom_cholesterol_and_triglyceride_regulating_preparations || tscore_bucket_prior_ntm || risk_untreated_early_menopause || injectable_experience_during_rx || risk_smoking_tobacco || risk_patient_parent_fractured_their_hip || frag_frac_prior_ntm || comorb_encounter_for_screening_for_malignant_neoplasms || speciality || region || ethnicity || comorb_disorders_of_lipoprotein_metabolism_and_other_lipidemias || concom_narcotics || gluco_prior_ntm || race || comorb_long_term_current_drug_therapy || comorb_personal_history_of_other_diseases_and_conditions || comorb_gastro_esophageal_reflux_disease || persistency_flag || concom_cephalosporins || concom_viral_vaccines || comorb_osteoporosis_without_current_pathological_fracture || concom_fluoroquinolones || concom_anaesthetics_general || dexa_during_rx || risk

#### Numerical columns summary

###### Getting a summary of numerical columns where we see the averages and other representatives of numerical data to help us with feature pre-processing later

In [10]:
if num_cols == []:
    display(Markdown('__NO NUMERICAL COLUMNS AVAILABLE__'))
else:
    num_desc = data[num_cols].describe().T
    num_desc.insert(loc=5,column='IQR',value = (num_desc['75%']-num_desc['25%']))
    num_desc.drop(['25%','50%','75%'],axis=1,inplace=True)
    
    num_desc['skewness'] = data[num_cols].skew()
    num_desc['kurtosis'] = data[num_cols].kurt()
    num_desc.insert(loc=0, column='columns', value=num_desc.index)
    num_desc.insert(loc=3, column = 'median', value=data[num_cols].median())
    print(num_desc.round(4))


                                 columns   count    mean  median     std  min  \
dexa_freq_during_rx  dexa_freq_during_rx  3424.0  3.0161     0.0  8.1365  0.0   
count_of_risks            count_of_risks  3424.0  1.2395     1.0  1.0949  0.0   

                     IQR    max  skewness  kurtosis  
dexa_freq_during_rx  3.0  146.0    6.8087   74.7584  
count_of_risks       2.0    7.0    0.8798    0.9005  


#### Categorical columns summary

###### Getting a summary of categorical columns where we see the distribution across different labels of the categorical features to help us with feature pre-processing later

In [11]:
cat_summ = pd.DataFrame(columns=['Variable','Category','Frequency','Percentage(%)'])
for col in cat_cols:
    cat_col_summary = pd.DataFrame(data[col].value_counts(dropna=False)).reset_index()
    cat_col_summary.columns = ['Category','Frequency']
    cat_col_summary['Percentage(%)'] = (cat_col_summary["Frequency"]/cat_col_summary["Frequency"].sum())*100
    cat_col_summary['Variable'] = col
    cat_summ = pd.concat([cat_summ, cat_col_summary],ignore_index = True)
display(cat_summ[['Variable','Category','Frequency','Percentage(%)']].round(4))

,Variable,Category,Frequency,Percentage(%)
0,risk_excessive_thinness,N,3357,98.0432
1,risk_excessive_thinness,Y,67,1.9568
2,risk_vitamin_d_insufficiency,N,1788,52.2196
3,risk_vitamin_d_insufficiency,Y,1636,47.7804
4,comorb_vitamin_d_deficiency,N,2331,68.0783
...,...,...,...,...
3591,risk_segment_during_rx,Unknown,1497,43.7208
3592,risk_segment_during_rx,HR_VHR,965,28.1834
3593,risk_segment_during_rx,VLR_LR,962,28.0958
3594,comorb_other_joint_disorder_not_elsewhere_clas...,N,2425,70.8236


In [12]:
max_cat = cat_summ.groupby('Variable').aggregate('max').reset_index()
max_cat.head()

,Variable,Category,Frequency,Percentage(%)
0,age,>75,1422,41.530374
1,change_risk_segment,Worsened,2229,65.099299
2,change_t_score,Worsened,1660,48.481308
3,comorb_disorders_of_lipoprotein_metabolism_and...,Y,1765,51.547897
4,comorb_dorsalgia,Y,2645,77.248832


In [13]:
min_cat = cat_summ.groupby('Variable').aggregate('min').reset_index()

In [15]:
min_max_cat = max_cat.merge(min_cat, on = 'Variable', how='left')
min_max_cat.drop(['Category_x','Frequency_x','Frequency_y'], axis=1, inplace=True)
min_max_cat['max_diff'] = (min_max_cat['Percentage(%)_x']-min_max_cat['Percentage(%)_y']).abs()
min_max_cat.head()
min_max_cat.drop(min_max_cat[min_max_cat.max_diff > 65].index, inplace=True)
min_max_cat.head()

,Variable,Percentage(%)_x,Category_y,Percentage(%)_y,max_diff
0,age,41.530374,55-65,4.760514,36.769860
1,change_risk_segment,65.099299,Improved,0.642523,64.456776
2,change_t_score,48.481308,Improved,2.745327,45.735981
3,comorb_disorders_of_lipoprotein_metabolism_and...,51.547897,N,48.452103,3.095794
4,comorb_dorsalgia,77.248832,N,22.751168,54.497664


In [ ]:
# cat_cols = min_max_cat.Variable.unique().tolist()
# cat_cols

# Data Preprocessing

In [16]:
# Check missing values
data.columns[data.isna().any()]
display(Markdown('__NO MISSING DATA __'))

__NO MISSING DATA __

In [17]:
identifier = ['ptid']
target = ['persistency_flag']

##### Filtering for independent categorical coumns

In [18]:
#Filtering for independent cat_cols

ind_cat_cols = list(set(cat_cols) - set(identifier) - set(target))

In [19]:
#Finding categorical features with two labels

cat_cols_2_label = list(data.columns[data.nunique()==2])
ind_cat_cols_2_label = list(set(ind_cat_cols).intersection(set(cat_cols_2_label)))

##### Categorical data cannot be passed directly through a classifier (although a decision tree based classifier can still give an estimate but better to encode and change them into numerical features)

##### As a rule of thumb, using one hot encoding for categorical variables with 2 labels and label encoding for rest of the categorical features with more than 2 labels (to avoid the curse of dimensionality)
##### One hot encoding also comes with a possibility of dummy data curse, since a label False is nothing but Not True and hence says the same information as the True flag. Keeping both in the data will increase collinearity. Hence such cases are removed from the data hile performing one hot encoding

In [20]:
filt = ind_cat_cols_2_label+identifier
dummy_data_2_label = pd.get_dummies(data[filt], columns = ind_cat_cols_2_label, drop_first=True, dtype = np.int64)
dummy_data_2_label.head()

,ptid,gluco_during_rx_Y,risk_poor_health_frailty_Y,risk_excessive_thinness_Y,risk_family_history_of_osteoporosis_Y,risk_vitamin_d_insufficiency_Y,comorb_vitamin_d_deficiency_Y,risk_type_1_insulin_dependent_diabetes_Y,risk_osteogenesis_imperfecta_Y,concom_cholesterol_and_triglyceride_regulating_preparations_Y,...,concom_anaesthetics_general_Y,risk_segment_prior_ntm_VLR_LR,risk_recurring_falls_Y,dexa_during_rx_Y,frag_frac_during_rx_Y,risk_hysterectomy_oophorectomy_Y,concom_anti_depressants_and_mood_stabilisers_Y,risk_untreated_chronic_hyperthyroidism_Y,risk_rheumatoid_arthritis_Y,comorb_other_joint_disorder_not_elsewhere_classified_Y
0,P701,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,P702,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,P703,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,P704,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,P705,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [21]:
ind_cat_cols_n_label = list(set(ind_cat_cols)-set(ind_cat_cols_2_label))

print(ind_cat_cols_n_label)

['change_risk_segment', 'age', 'race', 'speciality', 'region', 'ethnicity', 'change_t_score', 'risk_segment_during_rx', 'tscore_bucket_during_rx']


In [22]:
le = LabelEncoder()
le_data = pd.DataFrame()
for i in ind_cat_cols_n_label:
    le_data[i] = le.fit_transform(data[i])
le_data['ptid'] = data['ptid']

In [23]:
#Final encoded data (Excluding target variable)

data_enc = pd.DataFrame()
keep = ['ptid']+num_cols
data_enc = data[keep]
data_enc['persistency_flag'] = np.where(data['persistency_flag'] == 'Persistent', 1, data['persistency_flag'])
data_enc['persistency_flag'] = np.where(data_enc['persistency_flag'] == 'Non-Persistent', 0, data_enc['persistency_flag'])
data_enc = data_enc.merge(dummy_data_2_label, how='left').merge(le_data, how='left')
data_enc.shape

(3424, 65)

# Data Preparation

### Train-Validation-Test Split

##### Validation set is usuaaly the set on the basis of which multiple iterations are run in order to get at the best model/parameters. Test set is the set on which final performance measyrement is done

In [24]:
#Train-Validation-test split
ind = list(set(list(data_enc.columns)) - set(target))
split_perc = 0.30
X = data_enc[ind]
Y = data_enc[target]

X_train, X_test_val, Y_train, Y_test_val = train_test_split(X,Y, test_size = split_perc, random_state = 41)
X_val, X_test, Y_val, Y_test = train_test_split(X_test_val,Y_test_val, test_size = 0.50, random_state = 41)

print(colored("\nTrain-validation-test split has been performed\n", 'green', attrs = ['bold']))
print(colored("\nTraining-Validation/test split percentage:", 'magenta', attrs = ['bold']), colored("\t{}:{} (%)", 'blue', attrs = ['bold']).format(int((1-split_perc)*100), int(split_perc*100)))
print(colored("\nTotal Observations:", 'magenta', attrs = ['bold']), colored("\t{} ", 'blue', attrs=['bold']).format(X.shape[0]))
print(colored("\nTotal Independent Features:", 'magenta', attrs = ['bold']), colored("\t{} ", 'blue', attrs=['bold']).format(X.shape[1]))
print(colored("\Training Observations:", 'magenta', attrs = ['bold']), colored("\t{} ", 'blue', attrs=['bold']).format(X_train.shape[0]))
print(colored("\Validation Observations:", 'magenta', attrs = ['bold']), colored("\t{} ", 'blue', attrs=['bold']).format(X_val.shape[0]))
print(colored("\nTest Observations:", 'magenta', attrs = ['bold']), colored("\t{} ", 'blue', attrs=['bold']).format(X_test.shape[0]))


Train-validation-test split has been performed


Training-Validation/test split percentage: 	70:30 (%)

Total Observations: 	3424 

Total Independent Features: 	64 
\Training Observations: 	2396 
\Validation Observations: 	514 

Test Observations: 	514 


### Removing linear dependency between independent variables

##### Variance inflation factor is used to capture the linear dependency of independent factors on each other. A high VIF value implies the feature has dependency with more features/higher dependency with some features. The calculation is insipired by R-square value post regression line fit and hence the formula for VIF = 1/(1-R**2)

In [25]:
#Linear dependency through VIF

X_train_processed = copy.deepcopy(X_train)
X_train_VIF = X_train_processed.drop("ptid", axis=1, inplace=False)
exog_df = add_constant(X_train_VIF)
vifs = pd.Series([1/(1. - OLS(exog_df[col].values,
                              exog_df.loc[:, exog_df.columns!=col].values).fit().rsquared) for col in exog_df],
                 index = exog_df.columns,
                 name = 'VIF')
vifs = pd.DataFrame(vifs)
vifs.drop('const', axis = 0, inplace = True)
vifs = vifs['VIF'].where(vifs['VIF'] <= 15, 15)
vifs = pd.DataFrame(vifs)
vifs = vifs.sort_values(by=['VIF'],ascending = True)

##### Dropping the collinear features from the dataset

In [26]:
multicorr_vars = vifs['VIF'].loc[lambda X: X>=10].index.tolist()
multicorr_vars.remove('count_of_risks')
X_train_VIF.drop(multicorr_vars, axis =1, inplace = True)

#### Variability check

##### COnstant/Quasi-constant variables OR variables with close to 0 standard deviation usually do not show enough variation to give any useful insights into the effect on the response variable. Hence its preferred to have more standard deviation for features, upto an extent, provided no normalization of the data has been perfoemed

In [ ]:
def variability_check(df):
    df_desc = df.describe().loc['std'] == 0
    return df_desc[df_desc].index.tolist()

In [ ]:
selected_features =[]
selected_features_dict = {}
col_drop = []

zero_variance = variability_check(X_train_VIF)
if len(zero_variance) == 0:
    print(colored("\nNo quasi-constant features/no column with 0 std dev", "green", attrs = ['bold']))
else:
    display(Markdown('<span style = "color:darkred"><br><b>QUASI_CONSTANT FEATURES'))
    display(pd.DataFrame(zero_variance, columns = ["Features"]))
print(colored("Standard Deviation of the data:", 'magenta', attrs = ['bold']))
display(pd.DataFrame(X_train_VIF.describe().loc['std'].round(4)))

# Data Balancing

##### Imbalanced dataset ill bias the training of the model towards the class with hugher value count in the dataset. To avoid this, the dataset is first balanced with respect to the response variable and then training is run

In [ ]:
cat_summ[cat_summ.Variable == 'persistency_flag']

In [ ]:
Y_train.value_counts()

##### Over or Under sampling are two of the options for balancing a dataset. Here oversampling has been used for the following reasons - 
##### 1. Less data availability
##### 2. Less variation in data

In [ ]:
#Oversampling

label = "SMOTE"
smotesampler = SMOTE(random_state = 41)
X_train_bal, Y_train_bal= smotesampler.fit_resample(X_train_VIF, Y_train.astype('boolean'))

In [ ]:
feature_count = 10
selected_model_feat = {}

# Feature Importance/Selection

## XGBoost Classifier

##### Inclination towards tree based classifiers provided the dataset being categorically heavy

In [ ]:
X = X_train_bal
Y = Y_train_bal

xg = XGBClassifier()
xg_model = xg.fit(X,Y)
xg_coeff = pd.DataFrame(np.round(xg_model.feature_importances_,4)).abs()
df_xgb = pd.concat([pd.DataFrame(X.columns), xg_coeff], axis=1)
df_xgb.columns = ['Features','Importance']
df_xgb = df_xgb.sort_values(by = 'Importance', ascending = False).reset_index(drop=True)
display(df_xgb[['Features','Importance']].head(15).round(4))

selected_model_feat.update({"XGB": df_xgb.Features.tolist()[:feature_count]})

## Random Forest Classifier

In [ ]:
X_rf = X_train_bal
Y_rf = Y_train_bal

rf = RandomForestClassifier()
rf_model = rf.fit(X_rf,Y_rf)
rf_coeff = pd.DataFrame(np.round(rf_model.feature_importances_,4)).abs()
df_rf = pd.concat([pd.DataFrame(X.columns), rf_coeff], axis=1)
df_rf.columns = ['Features','Importance']
df_rf = df_rf.sort_values(by = 'Importance', ascending = False).reset_index(drop=True)
display(df_rf[['Features','Importance']].head(15).round(4))

selected_model_feat.update({"RF": df_rf.Features.tolist()[:feature_count]})

## Final feature list

In [ ]:
final_feat = list(set(selected_model_feat['XGB']) | set(selected_model_feat['RF']))
len(final_feat)

In [ ]:
print(colored("\nFinal feature list:", 'magenta', attrs = ['bold']), ' || '.join(final_feat))

In [ ]:
feature_final = ["gluco_record_prior_ntm","frag_frac_prior_ntm","dexa_during_rx","concom_viral_vaccines", "risk_segment_during_rx",
"ntm_speciality","age_bucket","concom_systemic_corticosteroids_plain",
"comorb_encntr_for_general_exam_w_o_complaint_susp_or_reprtd_dx","dexa_freq_during_rx",
"comorb_encounter_for_immunization","comorb_encounter_for_screening_for_malignant_neoplasms",
"region","tscore_bucket_during_rx","concom_viral_vaccines","comorb_long_term_current_drug_therapy","change_risk_segment"]

# Visual Representation

##### Post getting the list of final featres which seem to be important provided the response variable, we can manually plot and check potential patterns of each of the independent feature with the response variable

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
m=1
for i in feature_final:
    plt.figure(m+1)
    m+=1
    sns.histplot(binwidth=0.5, x=i, hue="persistency_flag", data=data, stat="percent", multiple="dodge", legend = True)

# Modelling

### UDFs to make iterative function calls easier and reusability possible

In [ ]:
def fit_function(model_name, x_train, y_train):
    if model_name == 'XGB':
        params = {'learning_rate':[0.15,0.10,0.05],
                  'max_depth':[2,3,4,5],
                  'gamma':[0.0,0.1,0.2,0.3]}
        classifier = XGBClassifier()
        random_search = RandomizedSearchCV(classifier, param_distributions = params, cv=3, n_iter = 5, scoring = 'roc_auc', verbose = 3)
        
        model_opt = random_search.fit(x_train, y_train)
        opt_params = model_opt.best_params_
        classifier = classifier.set_params(**opt_params)
        model = classifier.fit(x_train, y_train)
        y_prob_train = model.predict_proba(x_train)
        cv = cross_val_score(classifier, x_train, y_train, cv=3, scoring = 'f1_macro')
    if model_name == 'RF':
        params = {'n_estimators':[100,200,300],
                  'max_depth':[2,3,4,5],
                  'min_samples_split':[3,4,5,6,7],
                  'min_samples_leaf':[1,2,3]}
        classifier = RandomForestClassifier()
        random_search = RandomizedSearchCV(classifier, param_distributions = params, cv=3, n_iter = 5, scoring = 'roc_auc', verbose = 3)
        
        model_opt = random_search.fit(x_train, y_train)
        opt_params = model_opt.best_params_
        classifier = classifier.set_params(**opt_params)
        model = classifier.fit(x_train, y_train)
        y_prob_train = model.predict_proba(x_train)
        cv = cross_val_score(classifier, x_train, y_train, cv=3, scoring = 'f1_macro')
    return model, y_prob_train, cv

In [ ]:
def Opt_thres(target, predicted, p, r, threshold_roc):
    n = min(len(threshold_roc), len(p))
    p=p[0:n]
    r=r[0:n]
    threshold_roc = threshold_roc[0:n]
    i = np.arange(len(1-r))
    roc = pd.DataFrame({'tf':pd.Series(p-r, index=i), 'threshold' : pd.Series(threshold_roc, index=i)})
    roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:-1]]

    return list(roc_t['threshold'])

In [ ]:
def classification_summary(y_train, y_prob, pos_class):
    fpr, tpr, threshold_roc = metrics.roc_curve(y_train,y_prob,pos_label = pos_class)
    threshold_roc[0]=1
    roc_auc = metrics.auc(fpr,tpr)
    p,r,thresholds = precision_recall_curve(y_train,y_prob,pos_label = pos_class)
    optimal_threshold = OP(y_train, y_prob, tpr, (1-fpr), threshold_roc)[0]

    return p,r,thresholds,optimal_threshold, fpr, tpr, roc_auc, threshold_roc

In [ ]:
def roc_plot(fpr, tpr):
    if sum(tpr) > sum(fpr):
        roc_auc = metrics.auc(fpr,tpr)
    else:
        roc_auc = metrics.auc(tpr,fpr)
    roc_fig = go.Figure()
    roc_fig.add_trace(go.Scatter(x=fpr,y=tpr,mode = 'lines', line = dict(color='navy',width=2, dash = 'dash')))
    roc_fig.add_trace(go.Scatter(name = 'AUC - %0.2f' % roc_auc, y=tpr, x=fpr, fill = 'tonexty', mode = 'lines', line_color = 'magenta'))
    roc_fig.update_layout(title = {'text':'ROC', 'y':0.9, 'x':0.5}, width = 900, height = 700)
    roc.update_xaxes(title = '1-Specificity')
    roc.update_yaxes(title = 'Sensitivity')
    roc_fig.show()

## XGBoost Classifier

### Fitting the best model on the training data

In [ ]:
model_name = 'XGB'
model, y_proba_train, cvs = fit_function(model_name, X_train_bal[final_feat], Y_train_bal)

In [ ]:
p,r,thresholds, optimal_threshold, fpr, tpr, roc_auc, threshold_roc = classification_summary(Y_train_bal, y_proba_train[:,0], True)

## ROC Curve

##### More the are under the curve, better the model performance. Since we have less data coverage and even distribution, the model tries its best to capture the available patterns present in the data and so is the inclination towards low accuracy

In [ ]:
plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#XGB Final Results

Y_pred_train = np.where(y_proba_train[:,0] > optimal_threshold, True, False)
optimal_threshold
cm_xgb = confusion_matrix(Y_train_bal, Y_pred_train, labels = [True,False])
tn,fp,fn,tp = confusion_matrix(Y_train_bal, Y_pred_train, labels = [True,False]).ravel()

accuracy_xgb = round(accuracy_score(y_pred = Y_pred_train, y_true = Y_train_bal),2)
precision_xgb = round(precision_score(y_pred = Y_pred_train, y_true = Y_train_bal, pos_label = True),2)
recall_xgb = round(recall_score(y_pred = Y_pred_train, y_true = Y_train_bal, pos_label = True),2)
f1_score_xgb = round(2*precision_xgb*recall_xgb/(precision_xgb+recall_xgb),2)
specificity_xgb = round(tn/(tn+fp),2)

auc = metrics.roc_auc_score(Y_train_bal, Y_pred_train)

## XGBoost Final Results

In [ ]:
display(Markdown(""" <span style="color:darkgreen; font-style:italic; font-size:15px">  <b> Final Results of XGBoost Classifier   </b></span>"""))

print(colored("\nOptimal Threshold:",'magenta', attrs=['bold']),round(optimal_threshold,2))
print(colored("\nConfusion matrix:",'magenta', attrs=['bold']),cm_xgb)
print(colored("\nAccuracy:",'magenta', attrs=['bold']),accuracy_xgb)
print(colored("\Precision:",'magenta', attrs=['bold']),precision_xgb)
print(colored("\Recall:",'magenta', attrs=['bold']),recall_xgb)
print(colored("\nF1 Score:",'magenta', attrs=['bold']),f1_score_xgb)
print(colored("\nArea Under The Curve:",'magenta', attrs=['bold']),round(auc,2))

### Trying Undersampling with RF as previous model accuracy was low

In [ ]:
#Undersampling

from imblearn.under_sampling import NearMiss

label = "US - NM"
NMundersample = NearMiss(version=1, n_neighbors=3)
X_train_bal_RF, Y_train_bal_RF= NMundersample.fit_resample(X_train_VIF, Y_train.astype('boolean'))

In [ ]:
X_RF = X_train_bal_RF
Y_RF = Y_train_bal_RF

RF = RandomForestClassifier()
RF_model = rf.fit(X_RF,Y_RF)
RF_coeff = pd.DataFrame(np.round(RF_model.feature_importances_,4)).abs()
df_RF = pd.concat([pd.DataFrame(X.columns), RF_coeff], axis=1)
df_RF.columns = ['Features','Importance']
df_RF = df_RF.sort_values(by = 'Importance', ascending = False).reset_index(drop=True)
display(df_RF[['Features','Importance']].head(10).round(4))

final_features_RF = df_RF.Features.head(10)

In [ ]:
X_train_bal_RF = X_train_bal_RF[final_features_RF]

model_name = 'XGB'
model_RF, y_proba_train_RF, cvs_RF = fit_function(model_name, X_train_bal_RF, Y_train_bal_RF)

In [ ]:
p_RF,r_RF,thresholds_RF, optimal_threshold_RF, fpr_RF, tpr_RF, roc_auc_RF, threshold_roc_RF = classification_summary(Y_train_bal_RF, y_proba_train_RF[:,0], True)

In [ ]:
plt.plot(fpr_RF,tpr_RF)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#RF Final Results

Y_pred_train_RF = np.where(y_proba_train_RF[:,0] > optimal_threshold_RF, True, False)
print(optimal_threshold_RF)
cm_RF = confusion_matrix(Y_train_bal_RF, Y_pred_train_RF, labels = [True,False])
tn_RF,fp_RF,fn_RF,tp_RF = confusion_matrix(Y_train_bal_RF, Y_pred_train_RF, labels = [True,False]).ravel()
print(tn_RF,fp_RF,fn_RF,tp_RF)

accuracy_RF = round(accuracy_score(y_pred = Y_pred_train_RF, y_true = Y_train_bal_RF),2)
precision_RF = round(precision_score(y_pred = Y_pred_train_RF, y_true = Y_train_bal_RF, pos_label = True),2)
recall_RF = round(recall_score(y_pred = Y_pred_train_RF, y_true = Y_train_bal_RF, pos_label = True),2)
f1_score_RF = round(2*precision_RF*recall_RF/(precision_RF+recall_RF),2)
specificity_RF = round(tn_RF/(tn_RF+fp_RF),2)

In [ ]:
display(Markdown(""" <span style="color:darkgreen; font-style:italic; font-size:15px">  <b> Final Results of Random Forest Classifier   </b></span>"""))

print(colored("\nOptimal Threshold:",'magenta', attrs=['bold']),round(optimal_threshold_RF,2))
print(colored("\nConfusion matrix:",'magenta', attrs=['bold']),cm_RF)
print(colored("\nAccuracy:",'magenta', attrs=['bold']),accuracy_RF)
print(colored("\Precision:",'magenta', attrs=['bold']),precision_RF)
print(colored("\Recall:",'magenta', attrs=['bold']),recall_RF)
print(colored("\nF1 Score:",'magenta', attrs=['bold']),f1_score_RF)

### Going with XGBoost classifier provided its performance is better

In [ ]:
Y_pred_value = pd.DataFrame(Y_pred_train)
Y_pred_value.columns = ['predicted_persistency_flag']
Y_pred_value.loc[Y_pred_value['predicted_persistency_flag'] == True, 'predicted_persistency_flag'] = 'Persistent'
Y_pred_value.loc[Y_pred_value['predicted_persistency_flag'] == False, 'predicted_persistency_flag'] = 'Non-Persistent'
Y_pred_value.head()

In [ ]:
Y_train_bal.head()

In [ ]:
Y_train_value = pd.DataFrame(Y_train_bal)
Y_train_value = Y_train_value.astype('int64')
Y_train_value.columns = ['actual_persistency_flag']
Y_train_value.loc[Y_train_value['actual_persistency_flag'] == True, 'actual_persistency_flag'] = 'Persistent'
Y_train_value.loc[Y_train_value['actual_persistency_flag'] == False, 'actual_persistency_flag'] = 'Non-Persistent'
Y_train_value.head()

In [ ]:
# X_train_bal.concat(Y_train_bal, axis = 1).concat(predjaj, axis = 1).head()
export_data = pd.concat([X_train_bal,Y_train_value, Y_pred_value], axis=1)

In [ ]:
export_data.to_excel('Final_data.xlsx')